In [9]:
from zipfile import ZipFile
import boto3
import os

In [3]:
with ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

In [19]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))
            
zipf = zipfile.ZipFile('logs.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('logs/', zipf)
zipf.close()      

In [18]:
import shutil
shutil.rmtree('logs')

In [ ]:
import os
os.remove("DenseNet161Weights.h5")

In [17]:
bucket='tog-dataset-sagemaker' # Replace with your s3 bucket name
Filename = "inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
s3 = boto3.Session().resource('s3').Bucket(bucket).Object(Filename).download_file(Filename)

In [22]:
bucket='sn2020-final-project' # Replace with your s3 bucket name
file = 'InceptionWeights.h5'


url = 's3://{}/{}'.format(bucket, file)
boto3.Session().resource('s3').Bucket(bucket).Object(file).upload_file(file)
print('Done writing to {}'.format(url))

Done writing to s3://sn2020-final-project/InceptionWeights.h5


In [ ]:
# Models

model_airport_archaeological_site_path = currentPath + os.sep + "temp" + os.sep + "model_airport_archaeological_site" + os.sep + "DenseNet161Model.h5"
model_barn_crop_field_path = currentPath + os.sep + "temp" + os.sep + "model_barn_crop_field" + os.sep + "DenseNet161Model.h5"
model_dam_factory_path = currentPath + os.sep + "temp" + os.sep + "model_dam_factory" + os.sep + "DenseNet161Model.h5"
model_fire_station_golf_course_path = currentPath + os.sep + "temp" + os.sep + "model_fire_station_golf_course" + os.sep + "DenseNet161Model.h5"
model_ground_transport_interchange_path = currentPath + os.sep + "temp" + os.sep + "model_ground_transport_interchange" + os.sep + "DenseNet161Model.h5"
model_lake_nuclear_path = currentPath + os.sep + "temp" + os.sep + "model_lake_nuclear" + os.sep + "DenseNet161Model.h5"
model_office_place_of_workship_path = currentPath + os.sep + "temp" + os.sep + "model_office_place_of_workship" + os.sep + "DenseNet161Model.h5"
model_police_railway_bridge_path = currentPath + os.sep + "temp" + os.sep + "model_police_railway_bridge" + os.sep + "DenseNet161Model.h5"
model_recreational_facility_shopping_path = currentPath + os.sep + "temp" + os.sep + "model_recreational_facility_shopping" + os.sep + "DenseNet161Model.h5"
model_single_unit_stadium_path = currentPath + os.sep + "temp" + os.sep + "model_single-unit_stadium" + os.sep + "DenseNet161Model.h5"
model_storage_tower_path = currentPath + os.sep + "temp" + os.sep + "model_storage_tower" + os.sep + "DenseNet161Model.h5"
model_tunel_zoo_path = currentPath + os.sep + "temp" + os.sep + "model_tunel_zoo" + os.sep + "DenseNet161Model.h5"
model_group1 = currentPath + os.sep + "temp" + os.sep + "model_group1" + os.sep + "DenseNet161Model.h5"

In [ ]:
loadedModel1 = load_model(model_group1)
loadedModel2 = load_model(model_barn_crop_field_path)
loadedModel3 = load_model(model_ground_transport_interchange_path)



path = trainDB + os.sep + "1_airport" + os.sep
imagesPredict = [f for f in listdir(path) if isfile(join(path,f))]
# print(imagesPredict)


for file in imagesPredict:
    img = load_img(path + file, target_size=( 224, 224))
    tensorImage = img_to_array(img) /255.
    tensorImage = np.expand_dims(tensorImage, axis=0)
    prediction1 = loadedModel1.predict(tensorImage, batch_size = 1)
    prediction2 = loadedModel2.predict(tensorImage, batch_size = 1)
    prediction3 = loadedModel3.predict(tensorImage, batch_size = 1)

    
    prediction = np.concatenate((prediction1[0], prediction2[0], prediction3[0]))
    #print(prediction)
    index = np.where(prediction == max(prediction))[0]
    if (0 != index[0].size):
        print("file "+ file + " is: " + classList[int(index[0])])